<a href="https://colab.research.google.com/github/goguaD/finalProjectML/blob/main/DLinear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ditogogua","key":"33ff556f8487784b1a2aa019105cd547"}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
!unzip -o walmart-recruiting-store-sales-forecasting.zip -d walmart_data/

401 Client Error: Unauthorized for url: https://www.kaggle.com/api/v1/competitions/data/download-all/walmart-recruiting-store-sales-forecasting
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: walmart_data/features.csv.zip  
  inflating: walmart_data/sampleSubmission.csv.zip  
  inflating: walmart_data/stores.csv  
  inflating: walmart_data/test.csv.zip  
  inflating: walmart_data/train.csv.zip  


In [5]:
!pip install einops datasets tsai pandas numpy scikit-learn --quiet


In [39]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from pathlib import Path


torch.manual_seed(0)
np.random.seed(0)


DATA_DIR = Path('/content/walmart_data')

train_df    = pd.read_csv(DATA_DIR / 'train.csv')
features_df = pd.read_csv(DATA_DIR / 'features.csv')
stores_df   = pd.read_csv(DATA_DIR / 'stores.csv')

df = (train_df
      .merge(features_df, on=['Store', 'Date'], how='left')
      .merge(stores_df,   on='Store',      how='left'))



In [40]:
df['IsHoliday'] = df['IsHoliday_x'].astype(int)
df.drop(columns=['IsHoliday_x', 'IsHoliday_y'], inplace=True, errors='ignore')

In [41]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['Store', 'Date'])
df = df.fillna(method='ffill')

df['Weekly_Sales'] = df['Weekly_Sales'].clip(lower=0)
df['Week']         = df['Date'].dt.isocalendar().week.astype(int)
df['Year']         = df['Date'].dt.year.astype(int)

/tmp/ipython-input-41-3157990802.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')         # forward‑fill weather / CPI / etc.


In [42]:
feature_cols = [
    'Temperature', 'Fuel_Price', 'CPI', 'Unemployment',
    'IsHoliday', 'Week', 'Year',
    'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'
]
target_col   = 'Weekly_Sales'

df[feature_cols] = df[feature_cols].fillna(0)

In [43]:
STORE_ID   = 1
store_df   = df[df['Store'] == STORE_ID].copy()

In [44]:
X_raw = store_df[feature_cols].values.astype(np.float32)
y_raw = np.log1p(store_df[target_col].values.astype(np.float32))   # log target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_raw)

In [45]:
def make_sequences(X, y, input_len=12, output_len=1):
    Xs, ys = [], []
    for i in range(len(X) - input_len - output_len):
        Xs.append(X[i:i+input_len])
        ys.append(y[i+input_len:i+input_len+output_len])
    return np.asarray(Xs, dtype=np.float32), np.asarray(ys, dtype=np.float32)

INPUT_LEN  = 12
OUTPUT_LEN = 1

X_seq, y_seq = make_sequences(X_scaled, y_raw, INPUT_LEN, OUTPUT_LEN)

In [48]:
X_train, X_val, y_train, y_val = train_test_split(
    X_seq, y_seq, test_size=0.2, shuffle=False
)

train_ds = TensorDataset(
    torch.tensor(X_train, dtype=torch.float32),
    torch.tensor(y_train, dtype=torch.float32)
)
val_ds = TensorDataset(
    torch.tensor(X_val, dtype=torch.float32),
    torch.tensor(y_val, dtype=torch.float32)
)


train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl   = DataLoader(val_ds,   batch_size=32)

In [ ]:
class DLinear(nn.Module):
    def __init__(self, input_len, n_features, output_len):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(input_len * n_features, output_len)

    def forward(self, x):
        x = self.flatten(x)  # [batch, input_len, n_features] -> [batch, input_len * n_features]
        return self.linear(x)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = DLinear(
    input_len=INPUT_LEN,
    n_features=X_train.shape[2],
    output_len=OUTPUT_LEN
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
loss_fn   = nn.L1Loss()

In [52]:
EPOCHS = 30
for epoch in range(1, EPOCHS+1):
    model.train()
    running = 0
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        loss = loss_fn(model(xb), yb)
        loss.backward()
        optimizer.step()
        running += loss.item() * xb.size(0)
    train_loss = running / len(train_dl.dataset)

    model.eval()
    running = 0
    with torch.no_grad():
        for xb, yb in val_dl:
            xb, yb = xb.to(device), yb.to(device)
            running += loss_fn(model(xb), yb).item() * xb.size(0)
    val_loss = running / len(val_dl.dataset)

    print(f"Epoch {epoch:02d} | train L1: {train_loss:.4f} | val L1: {val_loss:.4f}")


Epoch 01 | train L1: 3.0910 | val L1: 1.3851
Epoch 02 | train L1: 1.2989 | val L1: 1.3794
Epoch 03 | train L1: 1.2981 | val L1: 1.3809
Epoch 04 | train L1: 1.2987 | val L1: 1.3850
Epoch 05 | train L1: 1.2987 | val L1: 1.3809
Epoch 06 | train L1: 1.2987 | val L1: 1.3852
Epoch 07 | train L1: 1.2995 | val L1: 1.3729
Epoch 08 | train L1: 1.2983 | val L1: 1.3715
Epoch 09 | train L1: 1.2992 | val L1: 1.3711
Epoch 10 | train L1: 1.2996 | val L1: 1.3723
Epoch 11 | train L1: 1.2995 | val L1: 1.3701
Epoch 12 | train L1: 1.2988 | val L1: 1.3698
Epoch 13 | train L1: 1.2985 | val L1: 1.3699
Epoch 14 | train L1: 1.2985 | val L1: 1.3771
Epoch 15 | train L1: 1.2992 | val L1: 1.3721
Epoch 16 | train L1: 1.2983 | val L1: 1.3772
Epoch 17 | train L1: 1.2995 | val L1: 1.3690
Epoch 18 | train L1: 1.2998 | val L1: 1.3719
Epoch 19 | train L1: 1.2995 | val L1: 1.3688
Epoch 20 | train L1: 1.2988 | val L1: 1.3683
Epoch 21 | train L1: 1.2985 | val L1: 1.3683
Epoch 22 | train L1: 1.2987 | val L1: 1.3711
Epoch 23 |

In [55]:
model.eval()
preds, actuals, weights = [], [], []
holiday_idx = feature_cols.index('IsHoliday')

with torch.no_grad():
    for xb, yb in val_dl:
        xb = xb.to(device)
        preds.append(model(xb).detach().cpu().tolist())
        actuals.append(yb.detach().cpu().tolist())
        weights.append(xb[:, -1, holiday_idx].detach().cpu().tolist())



preds    = np.expm1(np.concatenate(preds).flatten())
actuals  = np.expm1(np.concatenate(actuals).flatten())
weights  = np.where(np.concatenate(weights).flatten() == 1, 5, 1)

wmae = np.sum(weights * np.abs(preds - actuals)) / np.sum(weights)
print(f"\n✅  WMAE (Store {STORE_ID}): {wmae:,.2f}")


✅  WMAE (Store 1): 18,232.87


In [57]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 19.3 MB/s eta 0:00:00


In [60]:
import os

os.environ["MLFLOW_TRACKING_USERNAME"] = "goguaD"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "685c4f5b2a0c555f9136c60a8666661d952de9be"

In [61]:

mlflow.set_tracking_uri("https://dagshub.com/goguaD/finalProjectML.mlflow")
mlflow.set_experiment("walmart-sales")

<Experiment: artifact_location='mlflow-artifacts:/0df2743fae1f428cbe1ef62bd56be3ce', creation_time=1751821538135, experiment_id='0', last_update_time=1751821538135, lifecycle_stage='active', name='walmart-sales', tags={}>

In [63]:
import mlflow
import mlflow.pytorch


with mlflow.start_run():

    mlflow.log_param("input_len", input_len)
    mlflow.log_param("output_len", output_len)
    mlflow.log_param("learning_rate", 1e-3)
    mlflow.log_param("model_type", "DLinear")

    for epoch in range(10):
        model.train()
        for xb, yb in train_dl:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}, Train Loss: {loss.item():.4f}")

    model.eval()
    preds, actuals, weights = [], [], []
    for xb, yb in val_dl:
        with torch.no_grad():
            y_pred = model(xb)
        preds.extend(y_pred.detach().cpu().tolist())
        actuals.extend(yb.detach().cpu().tolist())
        weights.extend(xb[:, -1, features.index('IsHoliday')].detach().cpu().tolist())

    preds = np.expm1(np.array(preds))
    actuals = np.expm1(np.array(actuals))
    weights = np.array(weights)
    weights = np.where(weights == 1, 5, 1)
    wmae = np.sum(weights * np.abs(preds - actuals)) / np.sum(weights)
    print(f"\n✅ WMAE: {wmae:.4f}")

    mlflow.log_metric("WMAE", wmae)
    mlflow.end_run()


Epoch 1, Train Loss: 0.9557
Epoch 2, Train Loss: 1.4136
Epoch 3, Train Loss: 1.3063
Epoch 4, Train Loss: 2.1955
Epoch 5, Train Loss: 1.9252
Epoch 6, Train Loss: 1.4242
Epoch 7, Train Loss: 1.5899
Epoch 8, Train Loss: 1.4912
Epoch 9, Train Loss: 1.1488
Epoch 10, Train Loss: 1.0308

✅ WMAE: 37452165.1984
🏃 View run salty-cod-350 at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0/runs/e144eb5f7b6b46a0b64fbbb3fdfa52e7
🧪 View experiment at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0
